In [1]:
# ! pip install gensim
import gensim
import gensim.downloader as api

In [2]:
from human_ai.code.dataset import get_train_dataset,get_test_dataset 

In [3]:
data = get_train_dataset(test_size=0.15)
train_data, test_data = data['train'], data['test']

def read_corpus(data, tokens_only=False):
        for i, sample in enumerate(data):
            tokens = gensim.utils.simple_preprocess(sample['text'])
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])


Stringifying the column:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [4]:
train_corpus = list(read_corpus(train_data))
test_corpus = list(read_corpus(test_data))

In [5]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [6]:
model.build_vocab(train_corpus)

In [7]:
print(f"Word 'penalty' appeared {model.wv.get_vecattr('penalty', 'count')} times in the training corpus.")

Word 'penalty' appeared 32 times in the training corpus.


In [8]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [9]:
inferred_vector = model.infer_vector(test_corpus[0].words)

### Predict

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import csv
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import classification_report
from human_ai.code.evaluate import save_predictions
from human_ai.code.dataset import get_train_dataset

train_set = data['train']
test_set = data['test']

In [11]:
X_train = np.array([model.infer_vector(text.words) for text in train_corpus])

In [12]:
X_test = np.array([model.infer_vector(text.words) for text in test_corpus])

In [16]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(X_train, train_set['label'])
predictions = clf.predict(X_test)
accuracy_score(test_set['label'], predictions)

0.5383333333333333